In [1]:
import base64
import datetime as dt
import hmac
import requests
import pandas as pd
import os

In [ ]:
df = pd.read_excel('../data/wallet.xlsx')
list_cripto = df[["Unnamed: 25", "Unnamed: 26"]][5:]
list_cripto

In [ ]:
def send_signed_request(http_method, url_path, payload={}):
    def get_time():
        return dt.datetime.utcnow().isoformat()[:-3]+'Z'
    
    def signature(timestamp, method, request_path, body, secret_key):
        if str(body) == '{}' or str(body) == 'None':
            body = ''
        message = str(timestamp) + str.upper(method) + request_path + str(body)
        mac = hmac.new(bytes(secret_key, encoding='utf8'), bytes(message, encoding='utf-8'), digestmod='sha256')
        d = mac.digest()
        return base64.b64encode(d)
    
    # set request header
    def get_header(request='GET', endpoint='', body:dict=dict()):
        cur_time = get_time()
        header = dict()
        header['CONTENT-TYPE'] = 'application/json'
        header['OK-ACCESS-KEY'] = os.environ.get('OKX_API_KEY')
        header['OK-ACCESS-SIGN'] = signature(cur_time, request, endpoint, body, os.environ.get('OKX_API_SECRET'))
        header['OK-ACCESS-TIMESTAMP'] = str(cur_time)
        header['OK-ACCESS-PASSPHRASE'] = os.environ.get('OKX_PASS_PHRASE')
        return header

    url = 'https://www.okx.com' + url_path
    header = get_header(http_method, url_path, payload)
    response = requests.get(url, headers=header)
    return response.json()

send_signed_request("GET", "/api/v5/account/balance", payload={})